Statistics for protein and MFI data

Last Updated on: Nov 15, 2024

# Import Libraries

In [2]:
import sys

#Import libraries
#import scanorama
import time
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
from scipy.sparse import csr_matrix
import logging
#from skbio.stats.composition import clr

#import doubletdetection
import matplotlib.pyplot as plt
import matplotlib.axes as axes
#import os
#For CLR of ADTs
import scipy
import scipy.stats
from sklearn.preprocessing import scale
import os
import copy
import itertools
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from matplotlib.patches import Patch
from matplotlib.lines import Line2D


sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)


sc.logging.print_versions() 
sc.settings.set_figure_params(dpi=300)



-----
anndata     0.10.9
scanpy      1.10.3
-----
PIL                 10.4.0
asttokens           NA
backcall            0.2.0
backports           NA
colorama            0.4.6
comm                0.2.1
cycler              0.12.1
cython_runtime      NA
dateutil            2.9.0
debugpy             1.6.7
decorator           5.1.1
exceptiongroup      1.2.2
executing           0.8.3
h5py                3.11.0
igraph              0.11.6
importlib_resources NA
ipykernel           6.28.0
jaraco              NA
jedi                0.19.1
joblib              1.4.2
kiwisolver          1.4.7
legacy_api_wrap     NA
leidenalg           0.10.2
llvmlite            0.43.0
matplotlib          3.9.2
more_itertools      10.3.0
mpl_toolkits        NA
natsort             8.4.0
numba               0.60.0
numpy               2.0.2
packaging           24.1
pandas              2.2.3
parso               0.8.3
patsy               0.5.6
pexpect             4.8.0
pickleshare         0.7.5
pkg_resources       NA
pla

# Summary of statistical tests

1) MFI protein surface markers for WT versus SKG
Paired analysis: Paired t-test with BH for MTC
Unpaired analysis: Exact permutation test (no MTC needed) 

2) LN protein TRBV WT versus SKG
Paired analysis: Paired t-test with Benjamini-Hochberg (BH) for multiple testing correction (MTC)
Unpaired analysis: Exact permutation test (no MTC needed) 
Note: The MTC also included the data from the SKG High and SKG Low comparison for the paired VB14 because that sample only had 4 samples.

3) LN and joint protein LN SKG High versus SKG Low
All samples except Vb14
Paired analysis: Exact permutation test (no MTC needed) 
Unpaired analysis: Exact permutation test (no MTC needed) 

For Vb14 only 4 samples:
Paired analysis: Paired t-test with Benjamini-Hochberg (BH) for multiple testing correction (MTC), Unpaired analysis: Exact permutation test (no MTC needed) 
Note: MTC for the paired Vb14 done with the data from 5G

4) Vb freq SKG Zym versus PBS
Unpaired analysis: Exact permutation test (no MTC needed) 
For PBS naive versus mem
Paired analysis: Paired t-test with Benjamini-Hochberg (BH) for multiple testing correction (MTC)
All other paired analysis
Paired analysis: Exact permutation test (no MTC needed) 

5) MFI for GFP/Nr4a1 in TRBV enriched versus non-enriched TRBVs
LME with BH for MTC


# 1) MFI protein surface markers for WT versus SKG

In [21]:
import pandas as pd
from itertools import combinations
from scipy.stats import sem
import scipy.stats as stats
import statsmodels.stats.multitest

In [22]:
def unpaired_permutation_test(data_1, data_2):
    
    #Create array of all combinations
    all_data = list(data_1) + list(data_2)
    all_combinations = list(combinations(list(range(len(all_data))), len(data_1)))
    #Perform exact permutation test
    test_statistic = list()
    for combo in all_combinations:
        new_data_1 = [all_data[x] for x in combo]
        data_2_indices = set(list(range(len(all_data)))).difference(combo)
        new_data_2 = [all_data[x] for x in data_2_indices]
        diff_means = np.mean(new_data_1) - np.mean(new_data_2)
        test_statistic = test_statistic + [diff_means]
    #Calculate p value from exact permutation distribution
    sample_stat = np.mean(data_1) - np.mean(data_2)
    p_value = np.sum([abs(sample_stat) <= abs(x) for x in test_statistic])/len(all_combinations)
    return(sample_stat, p_value)

In [23]:
data_dic = {}
for marker in ['4-BB','CD69','OX-40','CD5']:
    data_dic[marker] = pd.read_csv('data/'+marker+'_YN_2024_05.csv', index_col = 0)
    print(data_dic[marker])

     GFPlo  GFPlo.1  GFPlo.2  GFPlo.3  GFPhi  GFPhi.1  GFPhi.2  GFPhi.3
WT   153.0    171.0    179.0      NaN  160.0    179.0    179.0      NaN
SKG  161.0    176.0    178.0    174.0  180.0    192.0    188.0    188.0
     GFPlo  GFPlo.1  GFPlo.2  GFPlo.3  GFPhi  GFPhi.1  GFPhi.2  GFPhi.3
WT   230.0    227.0    226.0      NaN  276.0    254.0    250.0      NaN
SKG  234.0    208.0    210.0    206.0  277.0    243.0    254.0    237.0
     GFPlo  GFPlo.1  GFPlo.2  GFPlo.3  GFPhi  GFPhi.1  GFPhi.2  GFPhi.3
WT    20.5     20.5     21.2      NaN   23.6     24.8     27.1      NaN
SKG   23.1     21.4     21.5     21.7   25.5     26.9     26.3     28.6
     GFPlo  GFPlo.1  GFPlo.2  GFPlo.3   GFPhi  GFPhi.1  GFPhi.2  GFPhi.3
WT   966.0    717.0    794.0      NaN  2653.0   1874.0   2170.0      NaN
SKG  201.0    166.0    167.0    152.0  1503.0   1226.0   1155.0   1166.0


In [24]:
pairings = [['paired','WT: GFPlo versus GFPhi', range(0,3),  range(4,7),0,0],
 ['paired','SKG: GFPlo versus GFPhi', range(0,4),  range(4,8), 1,1],
  ['unpaired','SKG GFPhi versus WT GFPhi', range(4,8), range(4,7), 1,0],
  ['unpaired','SKG GFPlo versus WT GFPlo', range(0,4),  range(0,3), 1,0]]

In [25]:
data_dic['4-BB'].iloc[0,range(0,3)]

GFPlo      153.0
GFPlo.1    171.0
GFPlo.2    179.0
Name: WT, dtype: float64

In [26]:
p_values = list()
effect_sizes = list()
index_list = list()
pair_list = list()
for marker in ['4-BB','CD69','OX-40','CD5']:
    df_1 = data_dic[marker]
    for pair in pairings:
        if pair[0] == 'paired':
            print(marker,pair[1], "parametric-paired")
            print(df_1.iloc[pair[4],pair[2]], df_1.iloc[pair[5],pair[3]])
            differences = np.array(df_1.iloc[pair[4],pair[2]]) - np.array(df_1.iloc[pair[5],pair[3]])
            p_value = stats.ttest_rel(df_1.iloc[pair[4],pair[2]], df_1.iloc[pair[5],pair[3]])[1]
            effect_size = np.mean(differences)/sem(differences)
            
        if pair[0] == 'unpaired':
            print(marker,pair[1], "unpaired permutation")
            print(df_1.iloc[pair[4],pair[2]], df_1.iloc[pair[5],pair[3]])
            effect_size, p_value = unpaired_permutation_test(df_1.iloc[pair[4],pair[2]], df_1.iloc[pair[5],pair[3]])
        effect_sizes = effect_sizes + [effect_size]
        p_values = p_values + [p_value]
        index_list = index_list + [marker + '_' +pair[1] ]
        pair_list = pair_list + [str(pair[0])]
        
result_df = pd.DataFrame(list(zip(pair_list, p_values,effect_sizes)), columns = ['paired','p_value','effect_size'], index = index_list)

4-BB WT: GFPlo versus GFPhi parametric-paired
GFPlo      153.0
GFPlo.1    171.0
GFPlo.2    179.0
Name: WT, dtype: float64 GFPhi      160.0
GFPhi.1    179.0
GFPhi.2    179.0
Name: WT, dtype: float64
4-BB SKG: GFPlo versus GFPhi parametric-paired
GFPlo      161.0
GFPlo.1    176.0
GFPlo.2    178.0
GFPlo.3    174.0
Name: SKG, dtype: float64 GFPhi      180.0
GFPhi.1    192.0
GFPhi.2    188.0
GFPhi.3    188.0
Name: SKG, dtype: float64
4-BB SKG GFPhi versus WT GFPhi unpaired permutation
GFPhi      180.0
GFPhi.1    192.0
GFPhi.2    188.0
GFPhi.3    188.0
Name: SKG, dtype: float64 GFPhi      160.0
GFPhi.1    179.0
GFPhi.2    179.0
Name: WT, dtype: float64
4-BB SKG GFPlo versus WT GFPlo unpaired permutation
GFPlo      161.0
GFPlo.1    176.0
GFPlo.2    178.0
GFPlo.3    174.0
Name: SKG, dtype: float64 GFPlo      153.0
GFPlo.1    171.0
GFPlo.2    179.0
Name: WT, dtype: float64
CD69 WT: GFPlo versus GFPhi parametric-paired
GFPlo      230.0
GFPlo.1    227.0
GFPlo.2    226.0
Name: WT, dtype: float64 G

In [27]:
result_df

,paired,p_value,effect_size
4-BB_WT: GFPlo versus GFPhi,paired,0.185312,-1.986799
4-BB_SKG: GFPlo versus GFPhi,paired,0.004362,-7.814741
4-BB_SKG GFPhi versus WT GFPhi,unpaired,0.057143,14.333333
4-BB_SKG GFPlo versus WT GFPlo,unpaired,0.571429,4.583333
CD69_WT: GFPlo versus GFPhi,paired,0.042509,-4.694159
CD69_SKG: GFPlo versus GFPhi,paired,0.001198,-12.159207
CD69_SKG GFPhi versus WT GFPhi,unpaired,0.571429,-7.250000
CD69_SKG GFPlo versus WT GFPlo,unpaired,0.142857,-13.166667
OX-40_WT: GFPlo versus GFPhi,paired,0.031876,-5.466266
OX-40_SKG: GFPlo versus GFPhi,paired,0.013755,-5.208641


In [28]:
result_df['paired']

4-BB_WT: GFPlo versus GFPhi          paired
4-BB_SKG: GFPlo versus GFPhi         paired
4-BB_SKG GFPhi versus WT GFPhi     unpaired
4-BB_SKG GFPlo versus WT GFPlo     unpaired
CD69_WT: GFPlo versus GFPhi          paired
CD69_SKG: GFPlo versus GFPhi         paired
CD69_SKG GFPhi versus WT GFPhi     unpaired
CD69_SKG GFPlo versus WT GFPlo     unpaired
OX-40_WT: GFPlo versus GFPhi         paired
OX-40_SKG: GFPlo versus GFPhi        paired
OX-40_SKG GFPhi versus WT GFPhi    unpaired
OX-40_SKG GFPlo versus WT GFPlo    unpaired
CD5_WT: GFPlo versus GFPhi           paired
CD5_SKG: GFPlo versus GFPhi          paired
CD5_SKG GFPhi versus WT GFPhi      unpaired
CD5_SKG GFPlo versus WT GFPlo      unpaired
Name: paired, dtype: object

In [29]:
result_df_unpaired = result_df.loc[[str(x) == 'unpaired' for x in result_df['paired']],:]
result_df_paired = result_df.loc[[str(x) == 'paired' for x in result_df['paired']],:]
result_df_paired['adj_p_value'] = statsmodels.stats.multitest.fdrcorrection(result_df_paired.iloc[:,1])[1]


/scratch/emccarthy/ipykernel_562271/3231014533.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df_paired['adj_p_value'] = statsmodels.stats.multitest.fdrcorrection(result_df_paired.iloc[:,1])[1]


In [30]:
result_df_unpaired.sort_values('p_value')

,paired,p_value,effect_size
CD5_SKG GFPhi versus WT GFPhi,unpaired,0.028571,-969.833333
CD5_SKG GFPlo versus WT GFPlo,unpaired,0.028571,-654.166667
4-BB_SKG GFPhi versus WT GFPhi,unpaired,0.057143,14.333333
OX-40_SKG GFPlo versus WT GFPlo,unpaired,0.057143,1.191667
CD69_SKG GFPlo versus WT GFPlo,unpaired,0.142857,-13.166667
OX-40_SKG GFPhi versus WT GFPhi,unpaired,0.257143,1.658333
CD69_SKG GFPhi versus WT GFPhi,unpaired,0.571429,-7.250000
4-BB_SKG GFPlo versus WT GFPlo,unpaired,0.571429,4.583333


In [ ]:
#result_df_unpaired.to_csv('results/WT_SKG_GFP_hi_and_lo_surface_protein_markers_073124_unpaired_results.csv')

In [31]:
result_df_paired.sort_values('adj_p_value')

,paired,p_value,effect_size,adj_p_value
CD69_SKG: GFPlo versus GFPhi,paired,0.001198,-12.159207,0.004790
CD5_SKG: GFPlo versus GFPhi,paired,0.000621,-15.175727,0.004790
4-BB_SKG: GFPlo versus GFPhi,paired,0.004362,-7.814741,0.011632
OX-40_SKG: GFPlo versus GFPhi,paired,0.013755,-5.208641,0.022009
CD5_WT: GFPlo versus GFPhi,paired,0.011739,-9.148204,0.022009
OX-40_WT: GFPlo versus GFPhi,paired,0.031876,-5.466266,0.042501
CD69_WT: GFPlo versus GFPhi,paired,0.042509,-4.694159,0.048582
4-BB_WT: GFPlo versus GFPhi,paired,0.185312,-1.986799,0.185312


In [ ]:
#result_df_paired.to_csv('results/WT_SKG_GFP_hi_and_lo_surface_protein_markers_080124_paired_results.csv')

# 2) LN protein TRBV WT versus SKG  and 3) LN and joint protein LN SKG High versus SKG Low

In [1]:
def paired_permutation_test(differences):
    #Define size of sample set
    index_list = list(range(len(differences)))
    #Create array of all combinations
    all_combinations = list()
    for i in range(len(index_list)+1):
        comb = combinations(index_list, i)
        all_combinations = all_combinations + list(comb)
    #Perform exact permutation test
    test_statistic = list()
    for combo in all_combinations:
        unchanged_indices = set(index_list).difference(combo)
        unchanged_diff = [differences[x] for x in unchanged_indices]
        changed_diff = [differences[x]*-1 for x in combo]
        new_differences = unchanged_diff + changed_diff
        if len(new_differences) != len(differences):
            print("error not length of original array")
            return
            
        diff_mean = np.mean(new_differences)
        diff_se = sem(new_differences)
        test_statistic = test_statistic + [diff_mean/diff_se]
    #Calculate p value from exact permutation distribution
    sample_stat = np.mean(differences)/sem(differences)
    p_value = np.sum([abs(sample_stat) <= abs(x) for x in test_statistic])/len(all_combinations)
    return(sample_stat, p_value)

In [2]:
def unpaired_permutation_test(data_1, data_2):
    
    #Create array of all combinations
    all_data = list(data_1) + list(data_2)
    all_combinations = list(combinations(list(range(len(all_data))), len(data_1)))
    #Perform exact permutation test
    test_statistic = list()
    for combo in all_combinations:
        new_data_1 = [all_data[x] for x in combo]
        data_2_indices = set(list(range(len(all_data)))).difference(combo)
        new_data_2 = [all_data[x] for x in data_2_indices]
        diff_means = np.mean(new_data_1) - np.mean(new_data_2)
        test_statistic = test_statistic + [diff_means]
    #Calculate p value from exact permutation distribution
    sample_stat = np.mean(data_1) - np.mean(data_2)
    p_value = np.sum([abs(sample_stat) <= abs(x) for x in test_statistic])/len(all_combinations)
    return(sample_stat, p_value)

In [5]:
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import sem
import scipy.stats as stats
LN_data = pd.read_csv('data/LN_protein_data.csv')

In [6]:
paired_analysis = [["WT_low","WT_high"],["SKG_low","SKG_high"]]
unpaired_analysis = [["WT_high","SKG_high"], ["WT_low","SKG_low"]]
results_dict = {}
results_dict_es = {}
for protein in LN_data.columns[2:]:
    p_values = list()
    effect_sizes = list()
    for pair in paired_analysis:
        differences = np.array(LN_data.loc[[x == pair[1] for x in LN_data['subgroup']], protein]) - np.array(LN_data.loc[[x == pair[0] for x in LN_data['subgroup']], protein])
        if len(differences) > 5:
            effect_size, p_value = paired_permutation_test(differences)
        else:
            print(protein,"parametric-paired")
            p_value = stats.ttest_rel(np.array(LN_data.loc[[x == pair[1] for x in LN_data['subgroup']], protein]), np.array(LN_data.loc[[x == pair[0] for x in LN_data['subgroup']], protein]))[1]
            effect_size = np.mean(differences)/sem(differences)
        effect_sizes = effect_sizes + [effect_size]
        p_values = p_values + [p_value]
        
    for pair in unpaired_analysis:
        data_1 = np.array(LN_data.loc[[x == pair[1] for x in LN_data['subgroup']], protein])
        data_2 = np.array(LN_data.loc[[x == pair[0] for x in LN_data['subgroup']], protein])
        if len(data_1) + len(data_2) > 5:
            effect_size, p_value = unpaired_permutation_test(data_1, data_2)
        else:
            print(protein,"non-parametric-un-paired")
            p_value = scipy.stats.ranksums(data_1, data_2)[1]
            effect_size = np.mean(data_1) - np.mean(data_2)
        effect_sizes = effect_sizes + [effect_size]
        p_values = p_values + [p_value]
    results_dict[protein] = p_values
    results_dict_es[protein] = effect_sizes
results_df = pd.DataFrame.from_dict(results_dict)
results_df_es = pd.DataFrame.from_dict(results_dict_es)

results_df.index = [x[1]+"-"+x[0] for x in paired_analysis+unpaired_analysis]
results_df_es.index = [x[1]+"-"+x[0] for x in paired_analysis+unpaired_analysis]

results_df = results_df[["Vb3","Vb5","Vb11","Vb6","Vb8","Vb14"]]
results_df_es = results_df_es[["Vb3","Vb5","Vb11","Vb6","Vb8","Vb14"]]

Vb5 parametric-paired
Vb5 parametric-paired
Vb3 parametric-paired
Vb3 parametric-paired
Vb6 parametric-paired
Vb6 parametric-paired
Vb8 parametric-paired
Vb8 parametric-paired
Vb11 parametric-paired
Vb11 parametric-paired
Vb14 parametric-paired
Vb14 parametric-paired


In [7]:
results_df

,Vb3,Vb5,Vb11,Vb6,Vb8,Vb14
WT_high-WT_low,0.006464,0.014754,0.001120,0.000885,0.000257,0.849426
SKG_high-SKG_low,0.009414,0.000467,0.001353,0.012679,0.002307,0.634852
SKG_high-WT_high,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571
SKG_low-WT_low,0.028571,0.028571,0.028571,0.942857,0.371429,0.028571


In [8]:
paired_analysis = [[1,2],[3,4],[5,6]]
unpaired_analysis = [[2,4],[2,6],[4,6],[1,3],[1,5],[3,5]]
results_dict = {}
results_dict_es = {}
for protein in LN_data.columns[2:]:
    LN_data_joint = pd.read_csv('data/' + protein + '_combo_joint.csv')
    p_values = list()
    effect_sizes = list()
    for pair in paired_analysis:
        differences = np.array(LN_data_joint.iloc[:,pair[1]]) - np.array(LN_data_joint.iloc[:,pair[0]])
        if len(differences) > 5:
            effect_size, p_value = paired_permutation_test(differences)
        else:
            print(protein,"parametric-paired")
            p_value = stats.ttest_rel(np.array(LN_data_joint.iloc[:,pair[1]]), np.array(LN_data_joint.iloc[:,pair[0]]))[1]
            effect_size = np.mean(differences)/sem(differences)
        effect_sizes = effect_sizes + [effect_size]
        p_values = p_values + [p_value]
        
    for pair in unpaired_analysis:
        data_1 = np.array(LN_data_joint.iloc[:,pair[1]])
        data_2 = np.array(LN_data_joint.iloc[:,pair[0]])
        if len(data_1) + len(data_2) > 5:
            effect_size, p_value = unpaired_permutation_test(data_1, data_2)
        else:
            print(protein,"non-parametric")
            p_value = scipy.stats.ranksums(data_1, data_2)[1]
            effect_size = np.mean(data_1) - np.mean(data_2)
        effect_sizes = effect_sizes + [effect_size]
        p_values = p_values + [p_value]
    results_dict[protein] = p_values
    results_dict_es[protein] = effect_sizes
results_df_joint = pd.DataFrame.from_dict(results_dict)
results_df_es_joint = pd.DataFrame.from_dict(results_dict_es)

results_df_joint.index = [LN_data_joint.columns[x[1]]+"-"+LN_data_joint.columns[x[0]] for x in paired_analysis+unpaired_analysis]
results_df_es_joint.index = [LN_data_joint.columns[x[1]]+"-"+LN_data_joint.columns[x[0]] for x in paired_analysis+unpaired_analysis]

results_df_joint = results_df_joint[["Vb3","Vb5","Vb11","Vb6","Vb8","Vb14"]]
results_df_es_joint = results_df_es_joint[["Vb3","Vb5","Vb11","Vb6","Vb8","Vb14"]]

Vb14 parametric-paired
Vb14 parametric-paired
Vb14 parametric-paired


In [9]:
results_df_joint

,Vb3,Vb5,Vb11,Vb6,Vb8,Vb14
SKG+zym (naive) GFPhi-SKG+zym (naive) GFPlo,0.015625,0.015625,0.015625,0.015625,0.015625,0.032312
SKG+zym (memory) GFPhi-SKG+zym (memory) GFPlo,0.062500,0.015625,0.015625,0.953125,0.046875,0.157427
SKG+zym (joint) GFPhi-SKG+zym (joint) GFPlo,0.015625,0.015625,0.015625,0.390625,0.062500,0.530986
SKG+zym (memory) GFPhi-SKG+zym (naive) GFPhi,0.077506,0.481935,0.000583,0.239510,0.625874,0.285714
SKG+zym (joint) GFPhi-SKG+zym (naive) GFPhi,0.000583,0.015152,0.298368,0.820513,0.405012,0.028571
SKG+zym (joint) GFPhi-SKG+zym (memory) GFPhi,0.000583,0.011072,0.001748,0.801865,0.309441,0.028571
SKG+zym (memory) GFPlo-SKG+zym (naive) GFPlo,0.003497,0.850816,0.003497,0.006993,0.152681,0.028571
SKG+zym (joint) GFPlo-SKG+zym (naive) GFPlo,0.000583,0.835082,0.006993,0.000583,0.000583,0.028571
SKG+zym (joint) GFPlo-SKG+zym (memory) GFPlo,0.062937,0.787879,0.165501,0.375874,0.002914,0.314286


In [10]:
import statsmodels.stats.multitest
adj_p_values = statsmodels.stats.multitest.fdrcorrection(list(results_df.iloc[0,:]) + list(results_df.iloc[1,:]) + list(results_df_joint.iloc[0:3,5]), alpha = 0.1)[1]


In [11]:
results_df.iloc[0,:] = adj_p_values[0:6]
results_df.iloc[1,:] = adj_p_values[6:12]
results_df_joint.iloc[0:3,5] = adj_p_values[12:]

In [12]:
results_df

,Vb3,Vb5,Vb11,Vb6,Vb8,Vb14
WT_high-WT_low,0.013852,0.022131,0.004058,0.004058,0.003504,0.849426
SKG_high-SKG_low,0.017650,0.003504,0.004058,0.021131,0.005767,0.680198
SKG_high-WT_high,0.028571,0.028571,0.028571,0.028571,0.028571,0.028571
SKG_low-WT_low,0.028571,0.028571,0.028571,0.942857,0.371429,0.028571


In [13]:
results_df_joint

,Vb3,Vb5,Vb11,Vb6,Vb8,Vb14
SKG+zym (naive) GFPhi-SKG+zym (naive) GFPlo,0.015625,0.015625,0.015625,0.015625,0.015625,0.044062
SKG+zym (memory) GFPhi-SKG+zym (memory) GFPlo,0.062500,0.015625,0.015625,0.953125,0.046875,0.196784
SKG+zym (joint) GFPhi-SKG+zym (joint) GFPlo,0.015625,0.015625,0.015625,0.390625,0.062500,0.612676
SKG+zym (memory) GFPhi-SKG+zym (naive) GFPhi,0.077506,0.481935,0.000583,0.239510,0.625874,0.285714
SKG+zym (joint) GFPhi-SKG+zym (naive) GFPhi,0.000583,0.015152,0.298368,0.820513,0.405012,0.028571
SKG+zym (joint) GFPhi-SKG+zym (memory) GFPhi,0.000583,0.011072,0.001748,0.801865,0.309441,0.028571
SKG+zym (memory) GFPlo-SKG+zym (naive) GFPlo,0.003497,0.850816,0.003497,0.006993,0.152681,0.028571
SKG+zym (joint) GFPlo-SKG+zym (naive) GFPlo,0.000583,0.835082,0.006993,0.000583,0.000583,0.028571
SKG+zym (joint) GFPlo-SKG+zym (memory) GFPlo,0.062937,0.787879,0.165501,0.375874,0.002914,0.314286


In [ ]:
#results_df.to_csv('results/TRBV_freq_protein_WT_v_SKG_092624.csv')

# 4) Vb freq SKG Zym versus PBS

In [14]:
protein_data = pd.read_csv('data/dLNPBS_+_Zym_all-Vb_freq_CD4-naive-memory_2024_04_05_clean.csv', sep = ',', index_col = 0).T
protein_data

,Vb6,Vb8,Vb14,Vb3,Vb5,Vb11
SKG + PBS LN naive,10.90,24.4,NaN,NaN,0.44,1.66
SKG + PBS LN naive.1,10.50,24.7,11.20,0.25,0.44,1.53
SKG + PBS LN naive.2,10.10,24.4,11.60,0.39,0.39,1.77
SKG + PBS LN naive.3,NaN,NaN,11.30,0.29,NaN,NaN
SKG + PBS LN naive.4,13.00,26.7,7.59,0.29,0.49,1.50
SKG + zym LN naive,10.70,25.3,12.10,0.31,0.49,1.81
SKG + zym LN naive.1,10.90,24.5,11.50,0.45,0.50,1.95
SKG + zym LN naive.2,10.40,25.1,11.40,0.28,0.50,1.83
SKG + zym LN naive.3,10.90,23.7,11.30,0.40,0.60,2.12
SKG + zym LN naive.4,13.60,26.7,8.29,0.30,0.58,2.11


In [15]:
#PBS naive
for protein in protein_data.columns:
    print(protein_data[protein].dropna()[0:4])

SKG + PBS LN naive      10.9
SKG + PBS LN naive.1    10.5
SKG + PBS LN naive.2    10.1
SKG + PBS LN naive.4    13.0
Name: Vb6, dtype: float64
SKG + PBS LN naive      24.4
SKG + PBS LN naive.1    24.7
SKG + PBS LN naive.2    24.4
SKG + PBS LN naive.4    26.7
Name: Vb8, dtype: float64
SKG + PBS LN naive.1    11.20
SKG + PBS LN naive.2    11.60
SKG + PBS LN naive.3    11.30
SKG + PBS LN naive.4     7.59
Name: Vb14, dtype: float64
SKG + PBS LN naive.1    0.25
SKG + PBS LN naive.2    0.39
SKG + PBS LN naive.3    0.29
SKG + PBS LN naive.4    0.29
Name: Vb3, dtype: float64
SKG + PBS LN naive      0.44
SKG + PBS LN naive.1    0.44
SKG + PBS LN naive.2    0.39
SKG + PBS LN naive.4    0.49
Name: Vb5, dtype: float64
SKG + PBS LN naive      1.66
SKG + PBS LN naive.1    1.53
SKG + PBS LN naive.2    1.77
SKG + PBS LN naive.4    1.50
Name: Vb11, dtype: float64


In [16]:
#PBS mem
for protein in protein_data.columns:
    print(protein_data[protein].dropna()[11:15])

SKG + PBS LN mem      8.14
SKG + PBS LN mem.1    8.12
SKG + PBS LN mem.2    9.15
SKG + PBS LN mem.4    4.84
Name: Vb6, dtype: float64
SKG + PBS LN mem      19.4
SKG + PBS LN mem.1    20.5
SKG + PBS LN mem.2    21.6
SKG + PBS LN mem.4    22.0
Name: Vb8, dtype: float64
SKG + PBS LN mem.1    10.70
SKG + PBS LN mem.2     7.86
SKG + PBS LN mem.3    10.40
SKG + PBS LN mem.4     5.66
Name: Vb14, dtype: float64
SKG + PBS LN mem.1    0.21
SKG + PBS LN mem.2    0.37
SKG + PBS LN mem.3    0.65
SKG + PBS LN mem.4    0.43
Name: Vb3, dtype: float64
SKG + PBS LN mem      0.31
SKG + PBS LN mem.1    0.38
SKG + PBS LN mem.2    0.33
SKG + PBS LN mem.4    0.44
Name: Vb5, dtype: float64
SKG + PBS LN mem      1.58
SKG + PBS LN mem.1    1.74
SKG + PBS LN mem.2    2.10
SKG + PBS LN mem.4    1.50
Name: Vb11, dtype: float64


In [17]:
results_dict_total = {}
results_dict_total_es = {}

for protein in protein_data.columns:
    protein_data_col = protein_data[protein].dropna()
    p_values = list()
    effect_sizes = list()
    differences = np.array(protein_data_col[0:4]) - np.array(protein_data_col[11:15])
    if len(differences) > 5:
            print(protein, "exact permutation")
            effect_size, p_value = paired_permutation_test(differences)
    else:
            print(protein,"parametric-paired")
            p_value = stats.ttest_rel(np.array(protein_data_col[0:4]), np.array(protein_data_col[11:15]))[1]
            effect_size = np.mean(differences)/sem(differences)
    effect_sizes = effect_sizes + [effect_size]
    p_values = p_values + [p_value]
        
    results_dict_total[protein] = p_values
    results_dict_total_es[protein] = effect_sizes
results_df_joint_total = pd.DataFrame.from_dict(results_dict_total)
results_df_es_joint_total = pd.DataFrame.from_dict(results_dict_total_es)

results_df_joint_total.index = ["PBS: Naive - Mem"]
results_df_es_joint_total.index = ["PBS: Naive - Mem"]

#results_df_joint = results_df_joint[["Vb3","Vb5","Vb11","Vb6","Vb8","Vb14"]]
#results_df_es_joint = results_df_es_joint[["Vb3","Vb5","Vb11","Vb6","Vb8","Vb14"]]

Vb6 parametric-paired
Vb8 parametric-paired
Vb14 parametric-paired
Vb3 parametric-paired
Vb5 parametric-paired
Vb11 parametric-paired


In [18]:
results_df_joint_total

,Vb6,Vb8,Vb14,Vb3,Vb5,Vb11
PBS: Naive - Mem,0.109638,0.003338,0.092172,0.320157,0.026979,0.309388


In [19]:
#Zym naive
for protein in protein_data.columns:
    print(protein_data[protein].dropna()[4:11])

SKG + zym LN naive      10.7
SKG + zym LN naive.1    10.9
SKG + zym LN naive.2    10.4
SKG + zym LN naive.3    10.9
SKG + zym LN naive.4    13.6
SKG + zym LN naive.5    11.9
SKG + zym LN naive.6    13.6
Name: Vb6, dtype: float64
SKG + zym LN naive      25.3
SKG + zym LN naive.1    24.5
SKG + zym LN naive.2    25.1
SKG + zym LN naive.3    23.7
SKG + zym LN naive.4    26.7
SKG + zym LN naive.5    25.7
SKG + zym LN naive.6    26.4
Name: Vb8, dtype: float64
SKG + zym LN naive      12.10
SKG + zym LN naive.1    11.50
SKG + zym LN naive.2    11.40
SKG + zym LN naive.3    11.30
SKG + zym LN naive.4     8.29
SKG + zym LN naive.5    10.00
SKG + zym LN naive.6     7.59
Name: Vb14, dtype: float64
SKG + zym LN naive      0.31
SKG + zym LN naive.1    0.45
SKG + zym LN naive.2    0.28
SKG + zym LN naive.3    0.40
SKG + zym LN naive.4    0.30
SKG + zym LN naive.5    0.40
SKG + zym LN naive.6    0.29
Name: Vb3, dtype: float64
SKG + zym LN naive      0.49
SKG + zym LN naive.1    0.50
SKG + zym LN naive

In [21]:
results_dict_p_value = {}
results_dict_es = {}

for protein in protein_data.columns:
    protein_data_col = protein_data[protein].dropna()
    p_values = list()
    effect_sizes = list()

    effect_size, p_value = unpaired_permutation_test(np.array(protein_data_col[0:4]), np.array(protein_data_col[4:11]))
    
    effect_sizes = effect_sizes + [effect_size]
    p_values = p_values + [p_value]
        
    results_dict_total[protein] = p_values
    results_dict_total_es[protein] = effect_sizes
    print(protein, "exact permutation")
results_df_joint_total_2 = pd.DataFrame.from_dict(results_dict_total)
results_df_es_joint_total_2 = pd.DataFrame.from_dict(results_dict_total_es)

results_df_joint_total_2.index = ["Naive: PBS - Zym"]
results_df_es_joint_total_2.index = ["Naive: PBS - Zym"]

Vb6 exact permutation
Vb8 exact permutation
Vb14 exact permutation
Vb3 exact permutation
Vb5 exact permutation
Vb11 exact permutation


In [22]:
results_df_joint_total.loc["Naive: PBS - Zym",:] = results_df_joint_total_2.iloc[0,:]
results_df_joint_total

,Vb6,Vb8,Vb14,Vb3,Vb5,Vb11
PBS: Naive - Mem,0.109638,0.003338,0.092172,0.320157,0.026979,0.309388
Naive: PBS - Zym,0.496970,0.672727,0.933333,0.266667,0.009091,0.027273


In [23]:
#Zym mem
for protein in protein_data.columns:
    print(protein_data[protein].dropna()[15:22])

SKG + zym LN mem      10.00
SKG + zym LN mem.1    14.10
SKG + zym LN mem.2     8.91
SKG + zym LN mem.3    10.50
SKG + zym LN mem.4     6.49
SKG + zym LN mem.5     6.97
SKG + zym LN mem.6     6.86
Name: Vb6, dtype: float64
SKG + zym LN mem      23.0
SKG + zym LN mem.1    23.4
SKG + zym LN mem.2    25.0
SKG + zym LN mem.3    25.1
SKG + zym LN mem.4    24.8
SKG + zym LN mem.5    23.4
SKG + zym LN mem.6    24.8
Name: Vb8, dtype: float64
SKG + zym LN mem      11.50
SKG + zym LN mem.1    13.00
SKG + zym LN mem.2    11.30
SKG + zym LN mem.3    12.40
SKG + zym LN mem.4     5.55
SKG + zym LN mem.5     9.99
SKG + zym LN mem.6     7.52
Name: Vb14, dtype: float64
SKG + zym LN mem      0.78
SKG + zym LN mem.1    0.75
SKG + zym LN mem.2    0.51
SKG + zym LN mem.3    0.78
SKG + zym LN mem.4    0.61
SKG + zym LN mem.5    0.78
SKG + zym LN mem.6    0.43
Name: Vb3, dtype: float64
SKG + zym LN mem      0.37
SKG + zym LN mem.1    0.28
SKG + zym LN mem.2    0.48
SKG + zym LN mem.3    0.29
SKG + zym LN mem.

In [25]:
results_dict_p_value = {}
results_dict_es = {}

for protein in protein_data.columns:
    protein_data_col = protein_data[protein].dropna()
    p_values = list()
    effect_sizes = list()

    effect_size, p_value = unpaired_permutation_test(np.array(protein_data_col[11:15]), np.array(protein_data_col[15:22]))
    
    effect_sizes = effect_sizes + [effect_size]
    p_values = p_values + [p_value]
        
    results_dict_total[protein] = p_values
    results_dict_total_es[protein] = effect_sizes
    print(protein, "exact permutation")
results_df_joint_total_3 = pd.DataFrame.from_dict(results_dict_total)
results_df_es_joint_total_3 = pd.DataFrame.from_dict(results_dict_total_es)

results_df_joint_total_3.index = ["Mem: PBS - Zym"]
results_df_es_joint_total_3.index = ["Mem: PBS - Zym"]

Vb6 exact permutation
Vb8 exact permutation
Vb14 exact permutation
Vb3 exact permutation
Vb5 exact permutation
Vb11 exact permutation


In [26]:
results_df_joint_total.loc["Mem: PBS - Zym",:] = results_df_joint_total_3.iloc[0,:]
results_df_joint_total

,Vb6,Vb8,Vb14,Vb3,Vb5,Vb11
PBS: Naive - Mem,0.109638,0.003338,0.092172,0.320157,0.026979,0.309388
Naive: PBS - Zym,0.496970,0.672727,0.933333,0.266667,0.009091,0.027273
Mem: PBS - Zym,0.345455,0.003030,0.357576,0.042424,0.300000,0.227273


In [36]:
results_dict_total = {}
results_dict_total_es = {}

for protein in protein_data.columns:
    protein_data_col = protein_data[protein].dropna()
    p_values = list()
    effect_sizes = list()
    differences = np.array(protein_data_col[4:11]) - np.array(protein_data_col[15:22])
    if len(differences) > 5:
            print(protein, "exact permutation")
            effect_size, p_value = paired_permutation_test(differences)
    else:
            print(protein,"parametric-paired")
            p_value = stats.ttest_rel(np.array(protein_data_col[4:11]), np.array(protein_data_col[15:22]))[1]
            effect_size = np.mean(differences)/sem(differences)
    effect_sizes = effect_sizes + [effect_size]
    p_values = p_values + [p_value]
        
    results_dict_total[protein] = p_values
    results_dict_total_es[protein] = effect_sizes
results_df_joint_total_4 = pd.DataFrame.from_dict(results_dict_total)
results_df_es_joint_total_4 = pd.DataFrame.from_dict(results_dict_total_es)

results_df_joint_total_4.index = ["Zym: Naive - Mem"]
results_df_es_joint_total_4.index = ["Zym: Naive - Mem"]

#results_df_joint = results_df_joint[["Vb3","Vb5","Vb11","Vb6","Vb8","Vb14"]]
#results_df_es_joint = results_df_es_joint[["Vb3","Vb5","Vb11","Vb6","Vb8","Vb14"]]

Vb6 exact permutation
Vb8 exact permutation
Vb14 exact permutation
Vb3 exact permutation
Vb5 exact permutation
Vb11 exact permutation


In [37]:
results_df_joint_total.loc["Zym: Naive - Mem",:] = results_df_joint_total_4.iloc[0,:]
results_df_joint_total

,Vb6,Vb8,Vb14,Vb3,Vb5,Vb11
PBS: Naive - Mem,0.109638,0.003338,0.092172,0.320157,0.026979,0.309388
Naive: PBS - Zym,0.496970,0.672727,0.933333,0.266667,0.009091,0.027273
Mem: PBS - Zym,0.345455,0.003030,0.357576,0.042424,0.300000,0.227273
Zym: Naive - Mem,0.140625,0.078125,0.765625,0.015625,0.187500,0.015625


In [38]:
#Zym joint
for protein in protein_data.columns:
    print(protein_data[protein].dropna()[22:])

SKG + zym joint (CD3+CD4+tot)      10.90
SKG + zym joint (CD3+CD4+tot).1    14.40
SKG + zym joint (CD3+CD4+tot).2     8.17
SKG + zym joint (CD3+CD4+tot).3     9.35
SKG + zym joint (CD3+CD4+tot).4     6.79
SKG + zym joint (CD3+CD4+tot).5     6.83
SKG + zym joint (CD3+CD4+tot).6     8.81
Name: Vb6, dtype: float64
SKG + zym joint (CD3+CD4+tot)      25.4
SKG + zym joint (CD3+CD4+tot).1    19.5
SKG + zym joint (CD3+CD4+tot).2    22.6
SKG + zym joint (CD3+CD4+tot).3    22.2
SKG + zym joint (CD3+CD4+tot).4    26.0
SKG + zym joint (CD3+CD4+tot).5    21.3
SKG + zym joint (CD3+CD4+tot).6    21.8
Name: Vb8, dtype: float64
SKG + zym joint (CD3+CD4+tot)       7.34
SKG + zym joint (CD3+CD4+tot).1    10.30
SKG + zym joint (CD3+CD4+tot).2     7.10
SKG + zym joint (CD3+CD4+tot).3    11.90
SKG + zym joint (CD3+CD4+tot).4     1.22
SKG + zym joint (CD3+CD4+tot).5     2.03
SKG + zym joint (CD3+CD4+tot).6     0.68
Name: Vb14, dtype: float64
SKG + zym joint (CD3+CD4+tot)      1.38
SKG + zym joint (CD3+CD4+to

In [39]:
results_dict_total = {}
results_dict_total_es = {}

for protein in protein_data.columns:
    protein_data_col = protein_data[protein].dropna()
    p_values = list()
    effect_sizes = list()
    differences = np.array(protein_data_col[4:11]) - np.array(protein_data_col[22:])
    if len(differences) > 5:
            print(protein, "exact permutation")
            effect_size, p_value = paired_permutation_test(differences)
    else:
            print(protein,"parametric-paired")
            p_value = stats.ttest_rel(np.array(protein_data_col[4:11]), np.array(protein_data_col[22:]))[1]
            effect_size = np.mean(differences)/sem(differences)
    effect_sizes = effect_sizes + [effect_size]
    p_values = p_values + [p_value]
        
    results_dict_total[protein] = p_values
    results_dict_total_es[protein] = effect_sizes
results_df_joint_total_5 = pd.DataFrame.from_dict(results_dict_total)
results_df_es_joint_total_5 = pd.DataFrame.from_dict(results_dict_total_es)

results_df_joint_total_5.index = ["Zym: Naive - Joint"]
results_df_es_joint_total_5.index = ["Zym: Naive - Joint"]

#results_df_joint = results_df_joint[["Vb3","Vb5","Vb11","Vb6","Vb8","Vb14"]]
#results_df_es_joint = results_df_es_joint[["Vb3","Vb5","Vb11","Vb6","Vb8","Vb14"]]

Vb6 exact permutation
Vb8 exact permutation
Vb14 exact permutation
Vb3 exact permutation
Vb5 exact permutation
Vb11 exact permutation


In [40]:
results_df_joint_total.loc["Zym: Naive - Joint",:] = results_df_joint_total_5.iloc[0,:]
results_df_joint_total

,Vb6,Vb8,Vb14,Vb3,Vb5,Vb11
PBS: Naive - Mem,0.109638,0.003338,0.092172,0.320157,0.026979,0.309388
Naive: PBS - Zym,0.496970,0.672727,0.933333,0.266667,0.009091,0.027273
Mem: PBS - Zym,0.345455,0.003030,0.357576,0.042424,0.300000,0.227273
Zym: Naive - Mem,0.140625,0.078125,0.765625,0.015625,0.187500,0.015625
Zym: Naive - Joint,0.125000,0.031250,0.031250,0.015625,0.109375,0.015625


In [41]:
results_dict_total = {}
results_dict_total_es = {}

for protein in protein_data.columns:
    protein_data_col = protein_data[protein].dropna()
    p_values = list()
    effect_sizes = list()
    differences = np.array(protein_data_col[15:22]) - np.array(protein_data_col[22:])
    if len(differences) > 5:
            print(protein, "exact permutation")
            effect_size, p_value = paired_permutation_test(differences)
    else:
            print(protein,"parametric-paired")
            p_value = stats.ttest_rel(np.array(protein_data_col[15:22]), np.array(protein_data_col[22:]))[1]
            effect_size = np.mean(differences)/sem(differences)
    effect_sizes = effect_sizes + [effect_size]
    p_values = p_values + [p_value]
        
    results_dict_total[protein] = p_values
    results_dict_total_es[protein] = effect_sizes
results_df_joint_total_6 = pd.DataFrame.from_dict(results_dict_total)
results_df_es_joint_total_6 = pd.DataFrame.from_dict(results_dict_total_es)

results_df_joint_total_6.index = ["Zym: Mem - Joint"]
results_df_es_joint_total_6.index = ["Zym: Mem - Joint"]

#results_df_joint = results_df_joint[["Vb3","Vb5","Vb11","Vb6","Vb8","Vb14"]]
#results_df_es_joint = results_df_es_joint[["Vb3","Vb5","Vb11","Vb6","Vb8","Vb14"]]

Vb6 exact permutation
Vb8 exact permutation
Vb14 exact permutation
Vb3 exact permutation
Vb5 exact permutation
Vb11 exact permutation


In [42]:
results_df_joint_total.loc["Zym: Mem - Joint",:] = results_df_joint_total_6.iloc[0,:]
results_df_joint_total

,Vb6,Vb8,Vb14,Vb3,Vb5,Vb11
PBS: Naive - Mem,0.109638,0.003338,0.092172,0.320157,0.026979,0.309388
Naive: PBS - Zym,0.496970,0.672727,0.933333,0.266667,0.009091,0.027273
Mem: PBS - Zym,0.345455,0.003030,0.357576,0.042424,0.300000,0.227273
Zym: Naive - Mem,0.140625,0.078125,0.765625,0.015625,0.187500,0.015625
Zym: Naive - Joint,0.125000,0.031250,0.031250,0.015625,0.109375,0.015625
Zym: Mem - Joint,0.609375,0.156250,0.015625,0.015625,0.046875,0.015625


In [46]:
#MTC

import statsmodels.stats.multitest
adj_p_values = statsmodels.stats.multitest.fdrcorrection(list(results_df_joint_total.iloc[0,:]), alpha = 0.1)[1]
adj_p_values

array([0.16445679, 0.02003048, 0.16445679, 0.32015718, 0.08093839,
       0.32015718])

In [49]:
results_df_joint_total.iloc[0,:] = adj_p_values

In [50]:
results_df_joint_total

,Vb6,Vb8,Vb14,Vb3,Vb5,Vb11
PBS: Naive - Mem,0.164457,0.020030,0.164457,0.320157,0.080938,0.320157
Naive: PBS - Zym,0.496970,0.672727,0.933333,0.266667,0.009091,0.027273
Mem: PBS - Zym,0.345455,0.003030,0.357576,0.042424,0.300000,0.227273
Zym: Naive - Mem,0.140625,0.078125,0.765625,0.015625,0.187500,0.015625
Zym: Naive - Joint,0.125000,0.031250,0.031250,0.015625,0.109375,0.015625
Zym: Mem - Joint,0.609375,0.156250,0.015625,0.015625,0.046875,0.015625


In [52]:
!pwd

/wynton/group/ye/emccarthy/EM/judy_proj/final_code


In [51]:
results_df_joint_total.to_csv('results/fig_6B_S7C_SKG_Zym_PBS_Vb_freq_092624.csv')

# 5) MFI for GFP/Nr4a1 in TRBV enriched versus non-enriched TRBVs

In [31]:
import statsmodels.formula.api as smf


In [32]:
test = pd.read_csv('data/PBS_v_Zym_all-Vb_GFP-MFI_dLN_joint_all_2024_04_05_clean.csv', sep = ',', index_col = 0).T
test

,Vb6,Vb8,Vb14,Vb3,Vb5,Vb11
SKG + PBS dLN - naive (Vb),2492.0,2608.0,NaN,NaN,4156.0,4303.0
SKG + PBS dLN - naive (Vb).1,2742.0,2801.0,2871.0,5429.0,4808.0,4544.0
SKG + PBS dLN - naive (Vb).2,2613.0,2685.0,2739.0,4058.0,4181.0,4416.0
SKG + PBS dLN - naive (Vb).3,NaN,NaN,2504.0,4866.0,NaN,NaN
SKG + PBS dLN - naive (Vb).4,1785.0,1916.0,1831.0,3133.0,2905.0,2964.0
SKG + zymosan dLN - naive (Vb),1942.0,2145.0,1920.0,4168.0,3534.0,3763.0
SKG + zymosan dLN - naive (Vb).1,1982.0,2258.0,2004.0,3443.0,3277.0,3562.0
SKG + zymosan dLN - naive (Vb).2,1917.0,2161.0,1955.0,3777.0,3013.0,3756.0
SKG + zymosan dLN - naive (Vb).3,2283.0,2132.0,2263.0,4555.0,3978.0,3982.0
SKG + zymosan dLN - naive (Vb).4,2582.0,2254.0,2290.0,4513.0,4120.0,3938.0


In [33]:
def lme_MFI_compartment(df):
    df_new = df.melt()
    df_new['mouse'] = [str(x) for x in list(range(len(df.index)))]*6
    df_new['tcr'] = ['non_enriched']*len(df.index)*3 + ['enriched']*len(df.index)*3
    my_model_fit = smf.mixedlm('value ~ tcr + mouse ', re_formula='1', groups='variable', data=df_new, missing='drop').fit()
    # get random effects
    my_model_fit.random_effects
    # get fixed effects
    print(my_model_fit.summary())
    return(my_model_fit.pvalues[1])

In [34]:
def lme_MFI_test(df):
    df_new = df.melt()
    df_new['mouse'] = [str(x) for x in list(range(len(df.index)))]*6
    df_new['tcr'] = ['non_enriched']*len(df.index)*3 + ['enriched']*len(df.index)*3
    print(df_new)

In [35]:
result_dict = {}

In [36]:
#PBS naive enriched versus non_enriched
df_current = test.iloc[0:5,:]
print(df_current)
lme_MFI_test(df_current)

                                 Vb6     Vb8    Vb14     Vb3     Vb5    Vb11
SKG + PBS dLN - naive (Vb)    2492.0  2608.0     NaN     NaN  4156.0  4303.0
SKG + PBS dLN - naive (Vb).1  2742.0  2801.0  2871.0  5429.0  4808.0  4544.0
SKG + PBS dLN - naive (Vb).2  2613.0  2685.0  2739.0  4058.0  4181.0  4416.0
SKG + PBS dLN - naive (Vb).3     NaN     NaN  2504.0  4866.0     NaN     NaN
SKG + PBS dLN - naive (Vb).4  1785.0  1916.0  1831.0  3133.0  2905.0  2964.0
   variable   value mouse           tcr
0       Vb6  2492.0     0  non_enriched
1       Vb6  2742.0     1  non_enriched
2       Vb6  2613.0     2  non_enriched
3       Vb6     NaN     3  non_enriched
4       Vb6  1785.0     4  non_enriched
5       Vb8  2608.0     0  non_enriched
6       Vb8  2801.0     1  non_enriched
7       Vb8  2685.0     2  non_enriched
8       Vb8     NaN     3  non_enriched
9       Vb8  1916.0     4  non_enriched
10     Vb14     NaN     0  non_enriched
11     Vb14  2871.0     1  non_enriched
12     Vb14  2739.

In [37]:
df_current = test.iloc[0:5,:]
group_1 = 'non_enriched_vb' 
group_2 = 'enriched_vb'
result_dict[df_current.index[0] + "_" + group_1 + "_versus_" + group_2] = lme_MFI_compartment(df_current)

                 Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      value     
No. Observations:       24           Method:                  REML      
No. Groups:             6            Scale:                   84067.3419
Min. group size:        4            Log-Likelihood:          -132.2458 
Max. group size:        4            Converged:               No        
Mean group size:        4.0                                             
------------------------------------------------------------------------
                      Coef.   Std.Err.    z    P>|z|   [0.025    0.975] 
------------------------------------------------------------------------
Intercept            4230.950  169.037  25.030 0.000  3899.643  4562.257
tcr[T.non_enriched] -1681.333  119.235 -14.101 0.000 -1915.029 -1447.637
mouse[T.1]            475.550  197.431   2.409 0.016    88.592   862.509
mouse[T.2]             58.384  197.431   0.296 0.767  -328.575   445.

/wynton/home/ye/emccarthy/miniconda3/envs/scanpy_092024/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/wynton/home/ye/emccarthy/miniconda3/envs/scanpy_092024/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/wynton/home/ye/emccarthy/miniconda3/envs/scanpy_092024/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/wynton/home/ye/emccarthy/miniconda3/envs/scanpy_092024/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/wynton/home/ye/emccarthy/miniconda3/envs/sca

In [38]:
#Zym naive enriched versus non_enriched
df_current = test.iloc[5:12,:]
print(df_current)
lme_MFI_test(df_current)

                                     Vb6     Vb8    Vb14     Vb3     Vb5  \
SKG + zymosan dLN - naive (Vb)    1942.0  2145.0  1920.0  4168.0  3534.0   
SKG + zymosan dLN - naive (Vb).1  1982.0  2258.0  2004.0  3443.0  3277.0   
SKG + zymosan dLN - naive (Vb).2  1917.0  2161.0  1955.0  3777.0  3013.0   
SKG + zymosan dLN - naive (Vb).3  2283.0  2132.0  2263.0  4555.0  3978.0   
SKG + zymosan dLN - naive (Vb).4  2582.0  2254.0  2290.0  4513.0  4120.0   
SKG + zymosan dLN - naive (Vb).5  2203.0  2619.0  2784.0  4711.0  3142.0   
SKG + zymosan dLN - naive (Vb).6  2145.0  2324.0  2416.0  4264.0  3824.0   

                                    Vb11  
SKG + zymosan dLN - naive (Vb)    3763.0  
SKG + zymosan dLN - naive (Vb).1  3562.0  
SKG + zymosan dLN - naive (Vb).2  3756.0  
SKG + zymosan dLN - naive (Vb).3  3982.0  
SKG + zymosan dLN - naive (Vb).4  3938.0  
SKG + zymosan dLN - naive (Vb).5  4700.0  
SKG + zymosan dLN - naive (Vb).6  4101.0  
   variable   value mouse           tcr
0      

In [39]:
df_current = test.iloc[5:12,:]
group_1 = 'non_enriched_vb' 
group_2 = 'enriched_vb'
result_dict[df_current.index[0] + "_" + group_1 + "_versus_" + group_2] = lme_MFI_compartment(df_current)

                 Mixed Linear Model Regression Results
Model:                 MixedLM      Dependent Variable:      value     
No. Observations:      42           Method:                  REML      
No. Groups:            6            Scale:                   65887.9186
Min. group size:       7            Log-Likelihood:          -247.8821 
Max. group size:       7            Converged:               Yes       
Mean group size:       7.0                                             
-----------------------------------------------------------------------
                      Coef.   Std.Err.   z    P>|z|   [0.025    0.975] 
-----------------------------------------------------------------------
Intercept            3758.238  167.504 22.437 0.000  3429.936  4086.540
tcr[T.non_enriched] -1692.476  193.107 -8.764 0.000 -2070.958 -1313.994
mouse[T.1]           -157.667  148.198 -1.064 0.287  -448.129   132.796
mouse[T.2]           -148.833  148.198 -1.004 0.315  -439.296   141.629
mouse[T.3

/scratch/emccarthy/ipykernel_2190555/810760929.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return(my_model_fit.pvalues[1])


In [40]:
#PBS memory enriched versus non_enriched
df_current = test.iloc[12:16,:]
print(df_current)
lme_MFI_test(df_current)

                                  Vb6     Vb8    Vb14     Vb3     Vb5    Vb11
SKG + PBS dLN - memory (Vb)    2828.0  3166.0     NaN     NaN  4341.0  2853.0
SKG + PBS dLN - memory (Vb).1  2610.0  2297.0  2277.0  4365.0  2983.0  2337.0
SKG + PBS dLN - memory (Vb).2  2702.0  2803.0  3127.0  4840.0  4963.0  3030.0
SKG + PBS dLN - memory (Vb).3     NaN     NaN  2234.0  3430.0     NaN     NaN
   variable   value mouse           tcr
0       Vb6  2828.0     0  non_enriched
1       Vb6  2610.0     1  non_enriched
2       Vb6  2702.0     2  non_enriched
3       Vb6     NaN     3  non_enriched
4       Vb8  3166.0     0  non_enriched
5       Vb8  2297.0     1  non_enriched
6       Vb8  2803.0     2  non_enriched
7       Vb8     NaN     3  non_enriched
8      Vb14     NaN     0  non_enriched
9      Vb14  2277.0     1  non_enriched
10     Vb14  3127.0     2  non_enriched
11     Vb14  2234.0     3  non_enriched
12      Vb3     NaN     0      enriched
13      Vb3  4365.0     1      enriched
14      Vb

In [41]:
df_current = test.iloc[12:16,:]
group_1 = 'non_enriched_vb' 
group_2 = 'enriched_vb'
result_dict[df_current.index[0] + "_" + group_1 + "_versus_" + group_2] = lme_MFI_compartment(df_current)

                 Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     value      
No. Observations:       18          Method:                 REML       
No. Groups:             6           Scale:                  144817.0399
Min. group size:        3           Log-Likelihood:         -103.5930  
Max. group size:        3           Converged:              Yes        
Mean group size:        3.0                                            
-----------------------------------------------------------------------
                      Coef.    Std.Err.   z    P>|z|   [0.025   0.975] 
-----------------------------------------------------------------------
Intercept             3996.936  432.089  9.250 0.000  3150.057 4843.814
tcr[T.non_enriched]  -1010.889  552.478 -1.830 0.067 -2093.727   71.949
mouse[T.1]            -679.991  257.422 -2.642 0.008 -1184.529 -175.454
mouse[T.2]              86.009  257.422  0.334 0.738  -418.529  590.546
mouse[T.3

/scratch/emccarthy/ipykernel_2190555/810760929.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return(my_model_fit.pvalues[1])


In [42]:
#Zym memory enriched versus non_enriched
df_current = test.iloc[17:24,:]
print(df_current)
lme_MFI_test(df_current)

                                      Vb6     Vb8    Vb14     Vb3     Vb5  \
SKG + zymosan dLN - memory (Vb)    1752.0  1733.0  1911.0  2886.0  2538.0   
SKG + zymosan dLN - memory (Vb).1  2524.0  2541.0  2538.0  2780.0  2459.0   
SKG + zymosan dLN - memory (Vb).2  1950.0  1968.0  2177.0  2130.0  2141.0   
SKG + zymosan dLN - memory (Vb).3  2723.0  2569.0  3426.0  3067.0  8172.0   
SKG + zymosan dLN - memory (Vb).4  2829.0  2483.0  2723.0  3376.0  5112.0   
SKG + zymosan dLN - memory (Vb).5  3281.0  3073.0  3888.0  2943.0  7759.0   
SKG + zymosan dLN - memory (Vb).6  2314.0  2712.0  2517.0  2385.0  6866.0   

                                     Vb11  
SKG + zymosan dLN - memory (Vb)    2165.0  
SKG + zymosan dLN - memory (Vb).1  2743.0  
SKG + zymosan dLN - memory (Vb).2  2474.0  
SKG + zymosan dLN - memory (Vb).3  3651.0  
SKG + zymosan dLN - memory (Vb).4  3408.0  
SKG + zymosan dLN - memory (Vb).5  3532.0  
SKG + zymosan dLN - memory (Vb).6  2920.0  
   variable   value mouse      

In [43]:
df_current = test.iloc[17:24,:]
group_1 = 'non_enriched_vb' 
group_2 = 'enriched_vb'
result_dict[df_current.index[0] + "_" + group_1 + "_versus_" + group_2] = lme_MFI_compartment(df_current)

                 Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     value      
No. Observations:       42          Method:                 REML       
No. Groups:             6           Scale:                  929775.8860
Min. group size:        7           Log-Likelihood:         -292.8168  
Max. group size:        7           Converged:              Yes        
Mean group size:        7.0                                            
-----------------------------------------------------------------------
                      Coef.    Std.Err.   z    P>|z|   [0.025   0.975] 
-----------------------------------------------------------------------
Intercept             2685.000  622.605  4.313 0.000  1464.716 3905.284
tcr[T.non_enriched]  -1041.667  713.880 -1.459 0.145 -2440.846  357.512
mouse[T.1]             433.333  556.709  0.778 0.436  -657.797 1524.464
mouse[T.2]             -24.167  556.709 -0.043 0.965 -1115.297 1066.964
mouse[T.3

/scratch/emccarthy/ipykernel_2190555/810760929.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return(my_model_fit.pvalues[1])


In [44]:
#Zym joint enriched versus non_enriched
df_current = test.iloc[-7:,:]
print(df_current)
lme_MFI_test(df_current)

                           Vb6     Vb8    Vb14      Vb3      Vb5    Vb11
SKG + zym joint (Vb)    5424.0  6077.0  5541.0   9015.0  10480.0  6163.0
SKG + zym joint (Vb).1  6288.0  4797.0  5325.0  10355.0   8478.0  7520.0
SKG + zym joint (Vb).2  6055.0  4681.0  5963.0   9316.0   8755.0  6339.0
SKG + zym joint (Vb).3  5191.0  5770.0  4865.0  10302.0  14851.0  6957.0
SKG + zym joint (Vb).4  5355.0  7339.0  2319.0   9446.0   7476.0  5859.0
SKG + zym joint (Vb).5  5018.0  4460.0  3101.0   6710.0   8942.0  6282.0
SKG + zym joint (Vb).6  5956.0  5222.0  4279.0  10330.0   7319.0  6288.0
   variable    value mouse           tcr
0       Vb6   5424.0     0  non_enriched
1       Vb6   6288.0     1  non_enriched
2       Vb6   6055.0     2  non_enriched
3       Vb6   5191.0     3  non_enriched
4       Vb6   5355.0     4  non_enriched
5       Vb6   5018.0     5  non_enriched
6       Vb6   5956.0     6  non_enriched
7       Vb8   6077.0     0  non_enriched
8       Vb8   4797.0     1  non_enriched
9     

In [45]:
df_current = test.iloc[-7:,:]
group_1 = 'non_enriched_vb' 
group_2 = 'enriched_vb'
result_dict[df_current.index[0] + "_" + group_1 + "_versus_" + group_2] = lme_MFI_compartment(df_current)

                  Mixed Linear Model Regression Results
Model:                 MixedLM      Dependent Variable:      value       
No. Observations:      42           Method:                  REML        
No. Groups:            6            Scale:                   1745363.4584
Min. group size:       7            Log-Likelihood:          -303.7631   
Max. group size:       7            Converged:               Yes         
Mean group size:       7.0                                               
-------------------------------------------------------------------------
                       Coef.    Std.Err.   z    P>|z|   [0.025    0.975] 
-------------------------------------------------------------------------
Intercept              8739.452  888.074  9.841 0.000  6998.859 10480.046
tcr[T.non_enriched]   -3245.571 1038.594 -3.125 0.002 -5281.178 -1209.965
mouse[T.1]               10.500  762.750  0.014 0.989 -1484.463  1505.463
mouse[T.2]             -265.167  762.750 -0.348 0.728 -1

/scratch/emccarthy/ipykernel_2190555/810760929.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return(my_model_fit.pvalues[1])


In [46]:
def lme_MFI_other(df,n1,n2):
    df_new = df.melt()
    df_new['mouse'] = [str(x) for x in list(range(len(df.index)))]*3
    df_new['group'] = ['PBS']*n1 + ['Zym']*n2 +['PBS']*n1 + ['Zym']*n2 + ['PBS']*n1 + ['Zym']*n2
    my_model_fit = smf.mixedlm('value ~ group ', re_formula='1', groups='variable', data=df_new, missing='drop').fit()
    # get random effects
    my_model_fit.random_effects
    # get fixed effects
    print(my_model_fit.summary())
    return(my_model_fit.pvalues[1])

In [47]:
def lme_MFI_other_test(df,n1,n2):
    df_new = df.melt()
    df_new['mouse'] = [str(x) for x in list(range(len(df.index)))]*3
    df_new['group'] = ['PBS']*n1 + ['Zym']*n2 +['PBS']*n1 + ['Zym']*n2 + ['PBS']*n1 + ['Zym']*n2
    print(df_new)

In [48]:
#Enriched TRBV Naive PBS versus Zym
df_current = test.iloc[0:12,3:]
print(df_current)
lme_MFI_other_test(df_current,5,7)

                                     Vb3     Vb5    Vb11
SKG + PBS dLN - naive (Vb)           NaN  4156.0  4303.0
SKG + PBS dLN - naive (Vb).1      5429.0  4808.0  4544.0
SKG + PBS dLN - naive (Vb).2      4058.0  4181.0  4416.0
SKG + PBS dLN - naive (Vb).3      4866.0     NaN     NaN
SKG + PBS dLN - naive (Vb).4      3133.0  2905.0  2964.0
SKG + zymosan dLN - naive (Vb)    4168.0  3534.0  3763.0
SKG + zymosan dLN - naive (Vb).1  3443.0  3277.0  3562.0
SKG + zymosan dLN - naive (Vb).2  3777.0  3013.0  3756.0
SKG + zymosan dLN - naive (Vb).3  4555.0  3978.0  3982.0
SKG + zymosan dLN - naive (Vb).4  4513.0  4120.0  3938.0
SKG + zymosan dLN - naive (Vb).5  4711.0  3142.0  4700.0
SKG + zymosan dLN - naive (Vb).6  4264.0  3824.0  4101.0
   variable   value mouse group
0       Vb3     NaN     0   PBS
1       Vb3  5429.0     1   PBS
2       Vb3  4058.0     2   PBS
3       Vb3  4866.0     3   PBS
4       Vb3  3133.0     4   PBS
5       Vb3  4168.0     5   Zym
6       Vb3  3443.0     6   Zym
7  

In [49]:
df_current = test.iloc[0:12,3:]
group_1 = 'PBS' 
group_2 = 'zym'
result_dict['Naive' + "_" + group_1 + "_versus_" + group_2] = lme_MFI_other(df_current,5,7)

            Mixed Linear Model Regression Results
Model:               MixedLM  Dependent Variable:  value      
No. Observations:    33       Method:              REML       
No. Groups:          3        Scale:               340691.2972
Min. group size:     11       Log-Likelihood:      -245.0716  
Max. group size:     11       Converged:           Yes        
Mean group size:     11.0                                     
--------------------------------------------------------------
               Coef.   Std.Err.   z    P>|z|  [0.025   0.975] 
--------------------------------------------------------------
Intercept     4146.917  206.632 20.069 0.000 3741.926 4551.907
group[T.Zym]  -236.393  211.221 -1.119 0.263 -650.379  177.593
variable Var 42916.971  130.882                               



/scratch/emccarthy/ipykernel_2190555/3302815876.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return(my_model_fit.pvalues[1])


In [50]:
#Enriched TRBV Memory PBS versus Zym
df_current = test.iloc[12:23,3:]
print(df_current)
lme_MFI_other_test(df_current,5,6)

                                      Vb3     Vb5    Vb11
SKG + PBS dLN - memory (Vb)           NaN  4341.0  2853.0
SKG + PBS dLN - memory (Vb).1      4365.0  2983.0  2337.0
SKG + PBS dLN - memory (Vb).2      4840.0  4963.0  3030.0
SKG + PBS dLN - memory (Vb).3      3430.0     NaN     NaN
SKG + PBS dLN - memory (Vb).4      2767.0  3033.0  2540.0
SKG + zymosan dLN - memory (Vb)    2886.0  2538.0  2165.0
SKG + zymosan dLN - memory (Vb).1  2780.0  2459.0  2743.0
SKG + zymosan dLN - memory (Vb).2  2130.0  2141.0  2474.0
SKG + zymosan dLN - memory (Vb).3  3067.0  8172.0  3651.0
SKG + zymosan dLN - memory (Vb).4  3376.0  5112.0  3408.0
SKG + zymosan dLN - memory (Vb).5  2943.0  7759.0  3532.0
   variable   value mouse group
0       Vb3     NaN     0   PBS
1       Vb3  4365.0     1   PBS
2       Vb3  4840.0     2   PBS
3       Vb3  3430.0     3   PBS
4       Vb3  2767.0     4   PBS
5       Vb3  2886.0     5   Zym
6       Vb3  2780.0     6   Zym
7       Vb3  2130.0     7   Zym
8       Vb3  306

In [51]:
df_current = test.iloc[12:23,3:]
group_1 = 'PBS' 
group_2 = 'zym'
result_dict['Memory' + "_" + group_1 + "_versus_" + group_2] = lme_MFI_other(df_current,5,6)

            Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  value       
No. Observations:   30       Method:              REML        
No. Groups:         3        Scale:               1950540.9598
Min. group size:    10       Log-Likelihood:      -246.2901   
Max. group size:    10       Converged:           Yes         
Mean group size:    10.0                                      
--------------------------------------------------------------
               Coef.    Std.Err.   z   P>|z|  [0.025   0.975] 
--------------------------------------------------------------
Intercept      3456.833  541.427 6.385 0.000 2395.655 4518.012
group[T.Zym]     61.833  520.489 0.119 0.905 -958.306 1081.972
variable Var 391795.888  436.062                              



/scratch/emccarthy/ipykernel_2190555/3302815876.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return(my_model_fit.pvalues[1])


In [52]:
def lme_MFI_zym(df, g1,g2):
    df_new = df.melt()
    df_new['mouse'] = [str(x) for x in list(range(7))]*6
    df_new['group'] = [g1]*7 + [g2]*7 + [g1]*7 + [g2]*7 + [g1]*7 + [g2]*7
    my_model_fit = smf.mixedlm('value ~ group + mouse ', re_formula='1', groups='variable', data=df_new, missing='drop').fit()
    # get random effects
    my_model_fit.random_effects
    # get fixed effects
    print(my_model_fit.summary())
    return(my_model_fit.pvalues[1])

In [53]:
def lme_MFI_zym_test(df, g1,g2):
    df_new = df.melt()
    df_new['mouse'] = [str(x) for x in list(range(7))]*6
    df_new['group'] = [g1]*7 + [g2]*7 + [g1]*7 + [g2]*7 + [g1]*7 + [g2]*7
    print(df_new)

In [54]:
#Enriched TRBV Zym Naive versus Mem
df_current = test.iloc[list(range(5,12))+list(range(17,24)),3:]
print(df_current)
lme_MFI_zym_test(df_current,group_1,group_2)

                                      Vb3     Vb5    Vb11
SKG + zymosan dLN - naive (Vb)     4168.0  3534.0  3763.0
SKG + zymosan dLN - naive (Vb).1   3443.0  3277.0  3562.0
SKG + zymosan dLN - naive (Vb).2   3777.0  3013.0  3756.0
SKG + zymosan dLN - naive (Vb).3   4555.0  3978.0  3982.0
SKG + zymosan dLN - naive (Vb).4   4513.0  4120.0  3938.0
SKG + zymosan dLN - naive (Vb).5   4711.0  3142.0  4700.0
SKG + zymosan dLN - naive (Vb).6   4264.0  3824.0  4101.0
SKG + zymosan dLN - memory (Vb)    2886.0  2538.0  2165.0
SKG + zymosan dLN - memory (Vb).1  2780.0  2459.0  2743.0
SKG + zymosan dLN - memory (Vb).2  2130.0  2141.0  2474.0
SKG + zymosan dLN - memory (Vb).3  3067.0  8172.0  3651.0
SKG + zymosan dLN - memory (Vb).4  3376.0  5112.0  3408.0
SKG + zymosan dLN - memory (Vb).5  2943.0  7759.0  3532.0
SKG + zymosan dLN - memory (Vb).6  2385.0  6866.0  2920.0
   variable   value mouse group
0       Vb3  4168.0     0   PBS
1       Vb3  3443.0     1   PBS
2       Vb3  3777.0     2   PBS
3 

In [55]:
df_current = test.iloc[list(range(5,12))+list(range(17,24)),3:]
group_1 = 'naive' 
group_2 = 'memory'
result_dict['Zym' + "_" + group_1 + "_versus_" + group_2] = lme_MFI_zym(df_current,group_1,group_2)

              Mixed Linear Model Regression Results
Model:                MixedLM   Dependent Variable:   value       
No. Observations:     42        Method:               REML        
No. Groups:           3         Scale:                1465463.7546
Min. group size:      14        Log-Likelihood:       -297.7439   
Max. group size:      14        Converged:            Yes         
Mean group size:      14.0                                        
------------------------------------------------------------------
                 Coef.    Std.Err.   z    P>|z|   [0.025   0.975] 
------------------------------------------------------------------
Intercept        3018.190  560.344  5.386 0.000  1919.937 4116.444
group[T.naive]    314.952  373.588  0.843 0.399  -417.267 1047.171
mouse[T.1]       -131.667  698.919 -0.188 0.851 -1501.523 1238.190
mouse[T.2]       -293.833  698.919 -0.420 0.674 -1663.690 1076.023
mouse[T.3]       1391.833  698.919  1.991 0.046    21.977 2761.690
mouse[T.4]

/scratch/emccarthy/ipykernel_2190555/2377315623.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return(my_model_fit.pvalues[1])


In [56]:
#Enriched TRBV Zym Mem versus Joint
df_current = test.iloc[list(range(17,24))+list(range(24,31)),3:]
print(df_current)
lme_MFI_zym_test(df_current,group_1,group_2)

                                       Vb3      Vb5    Vb11
SKG + zymosan dLN - memory (Vb)     2886.0   2538.0  2165.0
SKG + zymosan dLN - memory (Vb).1   2780.0   2459.0  2743.0
SKG + zymosan dLN - memory (Vb).2   2130.0   2141.0  2474.0
SKG + zymosan dLN - memory (Vb).3   3067.0   8172.0  3651.0
SKG + zymosan dLN - memory (Vb).4   3376.0   5112.0  3408.0
SKG + zymosan dLN - memory (Vb).5   2943.0   7759.0  3532.0
SKG + zymosan dLN - memory (Vb).6   2385.0   6866.0  2920.0
SKG + zym joint (Vb)                9015.0  10480.0  6163.0
SKG + zym joint (Vb).1             10355.0   8478.0  7520.0
SKG + zym joint (Vb).2              9316.0   8755.0  6339.0
SKG + zym joint (Vb).3             10302.0  14851.0  6957.0
SKG + zym joint (Vb).4              9446.0   7476.0  5859.0
SKG + zym joint (Vb).5              6710.0   8942.0  6282.0
SKG + zym joint (Vb).6             10330.0   7319.0  6288.0
   variable    value mouse   group
0       Vb3   2886.0     0   naive
1       Vb3   2780.0     1   n

In [57]:
df_current = test.iloc[list(range(17,24))+list(range(24,31)),3:]
group_1 = 'memory' 
group_2 = 'joint'
result_dict['Zym' + "_" + group_1 + "_versus_" + group_2] = lme_MFI_zym(df_current,group_1,group_2)

                Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    value       
No. Observations:      42         Method:                REML        
No. Groups:            3          Scale:                 2751245.7508
Min. group size:       14         Log-Likelihood:        -309.8369   
Max. group size:       14         Converged:             Yes         
Mean group size:       14.0                                          
---------------------------------------------------------------------
                   Coef.    Std.Err.   z    P>|z|   [0.025    0.975] 
---------------------------------------------------------------------
Intercept          7962.024  990.764  8.036 0.000  6020.162  9903.886
group[T.memory]   -4841.714  511.882 -9.459 0.000 -5844.985 -3838.444
mouse[T.1]          181.333  957.644  0.189 0.850 -1695.614  2058.281
mouse[T.2]         -348.667  957.644 -0.364 0.716 -2225.614  1528.281
mouse[T.3]         2292.167  957.644

/scratch/emccarthy/ipykernel_2190555/2377315623.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return(my_model_fit.pvalues[1])


In [58]:
#Enriched TRBV Zym Naive versus Joint
df_current = test.iloc[list(range(5,12))+list(range(24,31)),3:]
print(df_current)
lme_MFI_zym_test(df_current,group_1,group_2)

                                      Vb3      Vb5    Vb11
SKG + zymosan dLN - naive (Vb)     4168.0   3534.0  3763.0
SKG + zymosan dLN - naive (Vb).1   3443.0   3277.0  3562.0
SKG + zymosan dLN - naive (Vb).2   3777.0   3013.0  3756.0
SKG + zymosan dLN - naive (Vb).3   4555.0   3978.0  3982.0
SKG + zymosan dLN - naive (Vb).4   4513.0   4120.0  3938.0
SKG + zymosan dLN - naive (Vb).5   4711.0   3142.0  4700.0
SKG + zymosan dLN - naive (Vb).6   4264.0   3824.0  4101.0
SKG + zym joint (Vb)               9015.0  10480.0  6163.0
SKG + zym joint (Vb).1            10355.0   8478.0  7520.0
SKG + zym joint (Vb).2             9316.0   8755.0  6339.0
SKG + zym joint (Vb).3            10302.0  14851.0  6957.0
SKG + zym joint (Vb).4             9446.0   7476.0  5859.0
SKG + zym joint (Vb).5             6710.0   8942.0  6282.0
SKG + zym joint (Vb).6            10330.0   7319.0  6288.0
   variable    value mouse   group
0       Vb3   4168.0     0  memory
1       Vb3   3443.0     1  memory
2       Vb

In [59]:
df_current = test.iloc[list(range(5,12))+list(range(24,31)),3:]
group_1 = 'naive' 
group_2 = 'joint'
result_dict['Zym' + "_" + group_1 + "_versus_" + group_2] = lme_MFI_zym(df_current,group_1,group_2)


               Mixed Linear Model Regression Results
Model:                MixedLM    Dependent Variable:    value       
No. Observations:     42         Method:                REML        
No. Groups:           3          Scale:                 2050187.4212
Min. group size:      14         Log-Likelihood:        -304.3047   
Max. group size:      14         Converged:             Yes         
Mean group size:      14.0                                          
--------------------------------------------------------------------
                 Coef.    Std.Err.    z    P>|z|   [0.025    0.975] 
--------------------------------------------------------------------
Intercept        8450.548  755.411  11.187 0.000  6969.970  9931.125
group[T.naive]  -4526.762  441.878 -10.244 0.000 -5392.826 -3660.697
mouse[T.1]        -81.333  826.678  -0.098 0.922 -1701.592  1538.925
mouse[T.2]       -361.167  826.678  -0.437 0.662 -1981.425  1259.092
mouse[T.3]       1250.333  826.678   1.512 0.130  

/scratch/emccarthy/ipykernel_2190555/2377315623.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return(my_model_fit.pvalues[1])


In [60]:
result_dict

{'SKG + PBS dLN - naive (Vb)_non_enriched_vb_versus_enriched_vb': np.float64(3.742748575199215e-45),
 'SKG + zymosan dLN - naive (Vb)_non_enriched_vb_versus_enriched_vb': np.float64(1.876759020925808e-18),
 'SKG + PBS dLN - memory (Vb)_non_enriched_vb_versus_enriched_vb': np.float64(0.0672895952301949),
 'SKG + zymosan dLN - memory (Vb)_non_enriched_vb_versus_enriched_vb': np.float64(0.1445204599513768),
 'SKG + zym joint (Vb)_non_enriched_vb_versus_enriched_vb': np.float64(0.0017782493694746928),
 'Naive_PBS_versus_zym': np.float64(0.26306644503866616),
 'Memory_PBS_versus_zym': np.float64(0.9054349075774009),
 'Zym_naive_versus_memory': np.float64(0.3992019224800183),
 'Zym_memory_versus_joint': np.float64(3.1194709731528572e-21),
 'Zym_naive_versus_joint': np.float64(1.2543132829201083e-24)}

In [63]:
result_df = pd.DataFrame.from_dict(result_dict, orient = 'index')
result_df

,0
SKG + PBS dLN - naive (Vb)_non_enriched_vb_versus_enriched_vb,3.742749e-45
SKG + zymosan dLN - naive (Vb)_non_enriched_vb_versus_enriched_vb,1.876759e-18
SKG + PBS dLN - memory (Vb)_non_enriched_vb_versus_enriched_vb,6.728960e-02
SKG + zymosan dLN - memory (Vb)_non_enriched_vb_versus_enriched_vb,1.445205e-01
SKG + zym joint (Vb)_non_enriched_vb_versus_enriched_vb,1.778249e-03
Naive_PBS_versus_zym,2.630664e-01
Memory_PBS_versus_zym,9.054349e-01
Zym_naive_versus_memory,3.992019e-01
Zym_memory_versus_joint,3.119471e-21
Zym_naive_versus_joint,1.254313e-24


In [64]:
import statsmodels
result_df['adj_p_value'] = statsmodels.stats.multitest.fdrcorrection(result_df.iloc[:,0])[1]
result_df.columns = ['p_value', 'adj_p_value']
result_df

,p_value,adj_p_value
SKG + PBS dLN - naive (Vb)_non_enriched_vb_versus_enriched_vb,3.742749e-45,3.742749e-44
SKG + zymosan dLN - naive (Vb)_non_enriched_vb_versus_enriched_vb,1.876759e-18,4.691898e-18
SKG + PBS dLN - memory (Vb)_non_enriched_vb_versus_enriched_vb,6.728960e-02,1.121493e-01
SKG + zymosan dLN - memory (Vb)_non_enriched_vb_versus_enriched_vb,1.445205e-01,2.064578e-01
SKG + zym joint (Vb)_non_enriched_vb_versus_enriched_vb,1.778249e-03,3.556499e-03
Naive_PBS_versus_zym,2.630664e-01,3.288331e-01
Memory_PBS_versus_zym,9.054349e-01,9.054349e-01
Zym_naive_versus_memory,3.992019e-01,4.435577e-01
Zym_memory_versus_joint,3.119471e-21,1.039824e-20
Zym_naive_versus_joint,1.254313e-24,6.271566e-24
